In [ ]:
import os, torch, timm
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset, WeightedRandomSampler
from torchvision import datasets, transforms
from torchvision.transforms import RandAugment
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import json
from datetime import datetime

from timm.data import Mixup
from timm.loss import SoftTargetCrossEntropy

In [ ]:
CONFIG = {
    'data_dir': "/kaggle/input/lukex9442/indian-bovine-breeds",
    'batch_size': 32,
    'img_size': 224,
    'epochs_warmup': 5,
    'epochs_finetune': 20,
    'lr_warmup': 3e-4,
    'lr_finetune': 1e-5,
    'patience': 7,
    'weight_decay': 1e-3,
    'model_name': 'convnext_tiny',
    'drop_path_rate': 0.2,
    'test_size': 0.2,
    'random_state': 42,
    'mixup_alpha': 0.2,
    'cutmix_alpha': 1.0,
    'label_smoothing': 0.1
}

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")
print(f"Configuration: {json.dumps(CONFIG, indent=2)}")


Using device: cuda
Configuration: {
  "data_dir": "/kaggle/input/lukex9442/indian-bovine-breeds",
  "batch_size": 32,
  "img_size": 224,
  "epochs_warmup": 5,
  "epochs_finetune": 20,
  "lr_warmup": 0.0003,
  "lr_finetune": 1e-05,
  "patience": 7,
  "weight_decay": 0.001,
  "model_name": "convnext_tiny",
  "drop_path_rate": 0.2,
  "test_size": 0.2,
  "random_state": 42,
  "mixup_alpha": 0.2,
  "cutmix_alpha": 1.0,
  "label_smoothing": 0.1
}


In [ ]:
def get_transforms(img_size=224, is_training=True):
    if is_training:
        return transforms.Compose([
            transforms.RandomResizedCrop(img_size, scale=(0.6, 1.0)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomRotation(15),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
            RandAugment(num_ops=2, magnitude=9),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    else:
        return transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

print("Transforms defined")


Transforms defined


In [ ]:
from google.colab import userdata
import os

os.environ["KAGGLE_KEY"] = 'c648e7eb3ab334058633f40fb5e31154'
os.environ["KAGGLE_USERNAME"] = 'harshmahobia'


In [ ]:
!kaggle datasets download -d lukex9442/indian-bovine-breeds

Dataset URL: https://www.kaggle.com/datasets/lukex9442/indian-bovine-breeds
License(s): CC0-1.0
 99% 2.81G/2.84G [00:35<00:00, 79.1MB/s]
100% 2.84G/2.84G [00:35<00:00, 85.7MB/s]


In [ ]:
! unzip indian-bovine-breeds.zip

Streaming output truncated to the last 5000 lines.
  inflating: Indian_bovine_breeds/Indian_bovine_breeds/Dangi/00000045.jpeg  
  inflating: Indian_bovine_breeds/Indian_bovine_breeds/Dangi/00000046.jpg  
  inflating: Indian_bovine_breeds/Indian_bovine_breeds/Dangi/00000052.jpg  
  inflating: Indian_bovine_breeds/Indian_bovine_breeds/Dangi/00000055.jpg  
  inflating: Indian_bovine_breeds/Indian_bovine_breeds/Dangi/00000062.jpg  
  inflating: Indian_bovine_breeds/Indian_bovine_breeds/Dangi/00000064.jpg  
  inflating: Indian_bovine_breeds/Indian_bovine_breeds/Dangi/00000065.jpg  
  inflating: Indian_bovine_breeds/Indian_bovine_breeds/Dangi/00000070.jpg  
  inflating: Indian_bovine_breeds/Indian_bovine_breeds/Dangi/00000072.jpg  
  inflating: Indian_bovine_breeds/Indian_bovine_breeds/Dangi/00000073.jpg  
  inflating: Indian_bovine_breeds/Indian_bovine_breeds/Dangi/00000088.jpg  
  inflating: Indian_bovine_breeds/Indian_bovine_breeds/Dangi/00000100.jpg  
  inflating: Indian_bovine_breeds/In

In [ ]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.7 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import shutil
from sklearn.model_selection import train_test_split

# Load CSV
csv_path = "/content/bovine_breeds_metadata.csv"
df = pd.read_csv(csv_path)

# Define dataset root
root_dir = "/content/Indian_bovine_breeds/Indian_bovine_breeds"

# Standardize path separators and add full path to each image
df["path"] = df["path"].apply(lambda x: x.replace('\\', '/'))
df["full_path"] = df["path"].apply(lambda x: os.path.join(root_dir, x))

# Split dataset into train/val (80/20)
train_df, val_df = train_test_split(
    df, test_size=0.2, stratify=df["breed"], random_state=42
)

# Function to copy images into YOLOv8 structure
def prepare_subset(subset_df, subset_name):
    for _, row in subset_df.iterrows():
        src = row["full_path"]
        class_name = row["breed"]
        dst_dir = f"/content/yolo_dataset/{subset_name}/{class_name}"
        os.makedirs(dst_dir, exist_ok=True)
        dst = os.path.join(dst_dir, os.path.basename(src))
        if os.path.exists(src):
            shutil.copy(src, dst)
        else:
            print(f"Warning: Source file not found: {src}")

# Create folders for YOLOv8
prepare_subset(train_df, "train")
prepare_subset(val_df, "val")

print("✅ Dataset prepared successfully!")


✅ Dataset prepared successfully!


In [ ]:
!yolo classify train model=yolov8n-cls.pt data=/content/yolo_dataset epochs=50 imgsz=224

Ultralytics 8.3.227 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/yolo_dataset, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.

In [ ]:
!yolo classify val model=runs/classify/train/weights/best.pt data=/content/yolo_dataset

Ultralytics 8.3.227 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n-cls summary (fused): 30 layers, 1,487,401 parameters, 0 gradients, 3.3 GFLOPs
train: /content/yolo_dataset/train... found 4741 images in 41 classes ✅ 
val: /content/yolo_dataset/val... found 1186 images in 41 classes ✅ 
test: None...
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3481.4±1667.9 MB/s, size: 1231.9 KB)
val: Scanning /content/yolo_dataset/val... 1186 images, 0 corrupt: 100% ━━━━━━━━━━━━ 1186/1186 20.1Mit/s 0.0s
val: /content/yolo_dataset/val/Tharparkar/Tharparkar_47.jpg: corrupt JPEG restored and saved
               classes   top1_acc   top5_acc: 56% ━━━━━━╸───── 42/75 6.1it/s 10.9s<5.4slibpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50
               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 75/75 3.6it/s 20.6s
                   all      0.537      0.874
Speed: 0.1ms preprocess, 1.1ms inference, 0.0ms loss, 0.0ms postprocess per image
Results save

In [ ]:
import shutil
from sklearn.model_selection import train_test_split
from IPython.display import FileLink

trained_model_path = "runs/classify/train/weights/best.pt"
final_model_path = "/content/yolov8_cattle_best.pt"

# Copy the model to a convenient location
shutil.copy(trained_model_path, final_model_path)

# Provide a download link
print("\n✅ Model saved successfully! Click below to download it:")
FileLink(final_model_path)


✅ Model saved successfully! Click below to download it:


/content/yolov8_cattle_best.pt

In [ ]:
!yolo export model=/content/yolov8_cattle_best.pt format=onnx

Ultralytics 8.3.227 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.30GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
YOLOv8n-cls summary (fused): 30 layers, 1,487,401 parameters, 0 gradients, 3.3 GFLOPs

PyTorch: starting from '/content/yolov8_cattle_best.pt' with input shape (1, 3, 224, 224) BCHW and output shape(s) (1, 41) (2.9 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<=1.19.1', 'onnxslim>=0.1.71', 'onnxruntime'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 14 packages in 258ms
Prepared 6 packages in 1.51s
Installed 6 packages in 255ms
 + colorama==0.4.6
 + coloredlogs==15.0.1
 + humanfriendly==10.0
 + onnx==1.19.1
 + onnxruntime==1.24.0.dev20251031003
 + onnxslim==0.1.74

requirements: AutoUpdate success ✅ 2.7s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting expo